## Calculate Data Quality Score
**Introduction**: In this activity, you will calculate data quality scores for datasets using different metrics. You will explore examples where you assess completeness, accuracy, and consistency.

### Task 1: Completeness Score
1. Objective: Determine the percentage of non-missing values in a dataset.
2. Steps:
    - Load a sample dataset using Pandas.
    - Identify the columns with missing values.
    - Calculate the completeness score as the ratio of non-missing values to total values.
    - E.g., a dataset with customer information.

In [1]:
import pandas as pd
import numpy as np

# Step 1: Create sample dataset with missing values
data = {
    "Name": ["Alice", "Bob", None, "David", "Eve"],
    "Email": ["alice@example.com", None, "charlie@example.com", "david@example.com", "eve@example.com"],
    "Age": [25, 30, np.nan, 22, 45]
}

df = pd.DataFrame(data)

# Step 2: Identify columns with missing values
missing_counts = df.isnull().sum()
print("Missing values per column:\n", missing_counts)

# Step 3: Calculate completeness score per column (% of non-missing values)
completeness = (1 - (missing_counts / len(df))) * 100
print("\nCompleteness Score (% non-missing values per column):\n", completeness)

Missing values per column:
 Name     1
Email    1
Age      1
dtype: int64

Completeness Score (% non-missing values per column):
 Name     80.0
Email    80.0
Age      80.0
dtype: float64


### Task 2: Accuracy Score

1. Objective: Measure the accuracy of a dataset by comparing it against a reference dataset.
2. Steps:
    - Load the main dataset and a reference dataset.
    - Select key columns for accuracy check.
    - Match values from both datasets and calculate the accuracy percentage.
    - E.g., along existing dataset with sales information.

In [4]:
import pandas as pd
import numpy as np

# Sample main dataset (could be sales or user data)
main_data = {
    "CustomerID": [1, 2, 3, 4, 5],
    "Name": ["Alice", "Bob", "Charlie", "David", "Eve"],
    "Email": ["alice@example.com", "bob@example.com", "charlie@example.com", "david@example.com", "eve@example.com"],
    "SalesAmount": [100, 150, 200, 130, 170]
}

main_df = pd.DataFrame(main_data)

# Reference dataset (the "truth" to compare against)
reference_data = {
    "CustomerID": [1, 2, 3, 4, 5],
    "Name": ["Alice", "Bob", "Charlie", "David", "Eva"],  # Note: Eve is "Eva" here — intentional mismatch
    "Email": ["alice@example.com", "bob@example.com", "charlie@example.com", "david@example.com", "eve@example.com"],
    "SalesAmount": [100, 150, 210, 130, 170]  # Note: mismatch in SalesAmount for CustomerID=3
}

reference_df = pd.DataFrame(reference_data)

# Select key columns to check accuracy on
key_columns = ["Name", "Email", "SalesAmount"]

# Merge datasets on CustomerID (primary key)
merged_df = main_df.merge(reference_df, on="CustomerID", suffixes=('_main', '_ref'))

# Function to check if row values in key columns match exactly
def check_accuracy(row):
    for col in key_columns:
        main_val = row[f"{col}_main"]
        ref_val = row[f"{col}_ref"]
        if pd.isnull(main_val) and pd.isnull(ref_val):
            continue  # Treat missing values as matching
        if main_val != ref_val:
            return False
    return True

# Apply the accuracy check row-wise
merged_df['is_accurate'] = merged_df.apply(check_accuracy, axis=1)

# Calculate accuracy percentage
accuracy_percentage = merged_df['is_accurate'].mean() * 100

print("Merged Data with Accuracy Check:")
print(merged_df[['CustomerID'] + [f"{col}_main" for col in key_columns] + ['is_accurate']])
print(f"\nAccuracy Score: {accuracy_percentage:.2f}%")

Merged Data with Accuracy Check:
   CustomerID Name_main           Email_main  SalesAmount_main  is_accurate
0           1     Alice    alice@example.com               100         True
1           2       Bob      bob@example.com               150         True
2           3   Charlie  charlie@example.com               200        False
3           4     David    david@example.com               130         True
4           5       Eve      eve@example.com               170        False

Accuracy Score: 60.00%


### Task 3: Consistency Score

1. Objective: Evaluate the consistency within a dataset for specific columns.
2. Steps:
    - Choose a column expected to have consistent values.
    - Use statistical or rule-based checks to identify inconsistencies.
    - Calculate the consistency score by the ratio of consistent to total entries.
    - E.g., validating phone number formats in a contact list.

In [3]:

import pandas as pd
import re

# Step 1: Sample dataset with phone numbers
data = {
    "Name": ["Alice", "Bob", "Charlie", "David", "Eve"],
    "Phone": ["+1-202-555-0136", "202-555-0198", "5550136", "+1 202 555 0178", "202.555.0156"]
}

df = pd.DataFrame(data)

# Step 2: Define a phone number pattern (e.g., US phone number format with country code)
phone_pattern = re.compile(r"^\+1[- ]?\d{3}[- ]?\d{3}[- ]?\d{4}$")

# Step 3: Check each phone number for consistency
def is_consistent(phone):
    # Normalize phone string (remove dots)
    phone = phone.replace(".", "").strip()
    return bool(phone_pattern.match(phone))

df["is_consistent"] = df["Phone"].apply(is_consistent)

# Step 4: Calculate consistency score (% of consistent phone numbers)
consistency_score = df["is_consistent"].mean() * 100

print(df)
print(f"\nConsistency Score: {consistency_score:.2f}%")

      Name            Phone  is_consistent
0    Alice  +1-202-555-0136           True
1      Bob     202-555-0198          False
2  Charlie          5550136          False
3    David  +1 202 555 0178           True
4      Eve     202.555.0156          False

Consistency Score: 40.00%
